## Awesome Paper Dataframe by Crawling

<br>

### Development Environment

In [2]:
import re
import html
import requests
import pandas as pd
from glob import glob
from bs4 import BeautifulSoup
import pandas.io.formats.style

In [6]:
def make_page_soup(link):

    target = link
    response = requests.get(target)
    page = str(response.content)
    soup = BeautifulSoup(response.text, 'html.parser')
     
    return page, soup

In [384]:
def make_awesome_paper_df_type1(soup, link):

    awesome_paper_df = pd.DataFrame({'Title':[0],
                                'Journal/Conference':[0],
                                'Year':[0], 
                                'Author':[0],
                                'ID/Link':[0],
                                'Source':[0]})
    
    li = soup.select("li")
    li_texts = []
    for li_text in li:
        li_text = str(li_text)
        if 'class=' not in str(li_text) and 'href=' not in li_text:
            li_text = li_text[li_text.find('<li>')+4 : li_text.find("</li>")]
            li_texts.append(li_text)
        
    for idx, li_text in enumerate(li_texts):

        li_text_split = li_text.split('.')
    
        paper_title = li_text_split[0]
        paper_title = re.sub(r"\[.*?\]|\<.*?\>", "", paper_title)

        if ']' in paper_title and '[' not in paper_title:
            paper_title  = re.sub(r"].*?", "", paper_title)  

        if len(li_text_split) > 1:
            paper_author = li_text_split[1]
            if 'et al.' not in paper_author:
                paper_author = ""
        else:
            paper_author = ""

        paper_journal_conf = re.search(r'[A-Z ]+[0-9]+[0-9]+[0-9]+[0-9]', li_text)

        if paper_journal_conf != None:
            paper_journal_conf = paper_journal_conf.group().strip()
            if len(paper_journal_conf) == 4 and int(paper_journal_conf) > 2100:
                paper_journal_conf = ""
            else:
                paper_year = re.sub(r'[^0-9]', '', paper_journal_conf)     
        else:
            paper_journal_conf = ""
            paper_year = ""
        
        paper_id = re.search(r'[0-9]+[0-9]+[0-9]+[0-9.]+[0-9]+[0-9]+[0-9]+[0-9]+[0-9]', li_text)
        if paper_id != None:
            paper_id = paper_id.group().strip()
        else:
            paper_id = ""
    
        if len(paper_title) > 5:    
            awesome_paper_df.loc[idx] = [paper_title, 
                                    paper_journal_conf,
                                    paper_year, 
                                    paper_author,
                                    paper_id,
                                    link]
        
    return awesome_paper_df

In [385]:
def make_awesome_paper_df_type2(soup, link):

    awesome_paper_df = pd.DataFrame({'Title':[0],
                                'Journal/Conference':[0],
                                'Year':[0], 
                                'Author':[0],
                                'ID/Link':[0],
                                'Source':[0]})
    
    li = soup.select("li")
    li_texts = []
    for li_text in li:
        li_text = str(li_text)
        if '<p dir="auto">' in str(li_text) :
            li_text = li_text[li_text.find('<strong>')+8 : li_text.find("</strong>")]
            li_texts.append(li_text)
        
    for idx, li_text in enumerate(li_texts):

        li_text_split = li_text.split(',')
    
        paper_title = li_text_split[0]
        paper_author = ""

        paper_journal_conf = re.search(r'[A-Z ]+[0-9]+[0-9]+[0-9]+[0-9]', li_text)
        if paper_journal_conf != None:
            paper_journal_conf = paper_journal_conf.group().strip()
            if len(paper_journal_conf) == 4 and int(paper_journal_conf) > 2100:
                paper_journal_conf = ""
            else:
                paper_year = re.sub(r'[^0-9]', '', paper_journal_conf)
        else:
            paper_journal_conf = ""
            paper_year = ""
        

        paper_id = re.search(r'[0-9]+[0-9]+[0-9]+[0-9.]+[0-9]+[0-9]+[0-9]+[0-9]+[0-9]', li_text)
        if paper_id != None:
            paper_id = paper_id.group().strip()
        else:
            paper_id = ""

        if len(paper_title) > 5:    
            awesome_paper_df.loc[idx] = [paper_title, 
                                    paper_journal_conf,
                                    paper_year, 
                                    paper_author,
                                    paper_id,
                                    link]        
    return awesome_paper_df

In [400]:
def make_awesome_paper_df_type3(soup, link):

    awesome_paper_df = pd.DataFrame({'Title':[0],
                                'Journal/Conference':[0],
                                'Year':[0], 
                                'Author':[0],
                                'ID/Link':[0],
                                'Source':[0]})
    
    li = soup.select("li")
    li_texts = []
    for li_text in li:
        li_text = str(li_text)
        if 'class=' not in li_text and 'href=' in li_text:
            li_texts.append(li_text)
        
    for idx, li_text in enumerate(li_texts):

        paper_title = li_text[li_text.find('"nofollow">')+4 : li_text.find("</a>")]
        paper_title = paper_title[paper_title.find('">')+2 : ]

        li_text_split1 = li_text.split(",")[-1]
        li_text_split2 = li_text.split(",")[-2]

        if "href=" not in li_text_split1:
            paper_journal_conf = li_text_split1

        elif "href=" in li_text_split1:
            paper_journal_conf = li_text_split2

        paper_journal_conf = re.search(r'[A-Z ]+[0-9]+[0-9]+[0-9]+[0-9]', paper_journal_conf)
        if paper_journal_conf != None:
            paper_journal_conf = paper_journal_conf.group().strip()
            if len(paper_journal_conf) == 4 and int(paper_journal_conf) > 2100:
                paper_journal_conf = ""
            else:
                paper_year = re.sub(r'[^0-9]', '', paper_journal_conf)
        else:
            paper_journal_conf = ""
            paper_year = ""

        paper_author = ', '.join(li_text.split(",")[1:-1])
        paper_id = li_text[li_text.find('href=">')+11 : li_text.find('" rel="')]
        paper_id = paper_id[paper_id.find('="')+2 :]

        if "github" not in paper_title and len(paper_title) > 5:
            awesome_paper_df.loc[idx] = [paper_title, 
                                    paper_journal_conf,
                                    paper_year, 
                                    paper_author,
                                    paper_id,
                                    link]
        
    return awesome_paper_df

In [387]:
def make_awesome_paper_df(awesome_paper_df_list):

    awesome_paper_df = pd.concat(awesome_paper_df_list)
    awesome_paper_df.drop_duplicates(subset='Title', inplace=True)
    awesome_paper_df = awesome_paper_df.sort_values('Year', ascending=True).reset_index().drop(['index'], axis='columns')

    return awesome_paper_df

In [388]:
def make_awesome_nlp_paper_df(awesome_paper_df, conference_list):

    awesome_nlp_paper_df_list = []

    for conference in conference_list:
        awesome_nlp_paper_df = awesome_paper_df[(awesome_paper_df['Journal/Conference'].str.contains(conference))]
        awesome_nlp_paper_df_list.append(awesome_nlp_paper_df)

    awesome_nlp_paper_df = pd.concat(awesome_nlp_paper_df_list)
    awesome_nlp_paper_df.drop_duplicates(subset='Title', inplace=True)
    awesome_nlp_paper_df = awesome_nlp_paper_df.sort_values('Year', ascending=True).reset_index().drop(['index'], axis='columns')

    return awesome_nlp_paper_df

In [404]:
link = "https://github.com/FLHonker/Awesome-Knowledge-Distillation"
page, soup = make_page_soup(link)
awesome_paper_df1 = make_awesome_paper_df_type1(soup, link)

In [405]:
link = "https://github.com/DefangChen/Knowledge-Distillation-Paper"
page, soup = make_page_soup(link)
awesome_paper_df2 = make_awesome_paper_df_type2(soup, link)

In [406]:
link = "https://github.com/lhyfst/knowledge-distillation-papers"
page, soup = make_page_soup(link)
awesome_paper_df3 = make_awesome_paper_df_type3(soup, link)

In [407]:
awesome_paper_df_list = [awesome_paper_df1, awesome_paper_df2, awesome_paper_df3]
awesome_paper_df = make_awesome_paper_df(awesome_paper_df_list)

In [408]:
conference_list = ['NeurlPS', 'ICML', 'ICLR', 'AAAI', 'IJCAI', 'ACM', 'IEEE', 'IJCAI', 'AISTATS',
                    'ACL', 'EMNLP', 'NAACL', 'EACL', 'AACL', 'COLING', 'WMT', 'IJCNLP', 'HLT']
awesome_nlp_paper_df = make_awesome_nlp_paper_df(awesome_paper_df, conference_list)

In [409]:
pd.set_option('display.max_colwidth', None)
awesome_paper_df

,Title,Journal/Conference,Year,Author,ID/Link,Source
0,Distilling the knowledge in a neural network,,,,1503.02531,https://github.com/FLHonker/Awesome-Knowledge-Distillation
1,Distilled Hierarchical Neural Ensembles with Adaptive Inference Cost,,,,2003.01474,https://github.com/FLHonker/Awesome-Knowledge-Distillation
2,Distilling Knowledge from Ensembles of Acoustic Models for Joint CTC-Attention End-to-End Speech Recognition,,,,2005.09310,https://github.com/FLHonker/Awesome-Knowledge-Distillation
3,Collaborative Learning for Faster StyleGAN Embedding,,,,2007.01758,https://github.com/FLHonker/Awesome-Knowledge-Distillation
4,Group Knowledge Transfer: Collaborative Training of Large CNNs on the Edge,,,,2007.14513,https://github.com/FLHonker/Awesome-Knowledge-Distillation
...,...,...,...,...,...,...
688,Adaptive Multi-Teacher Knowledge Distillation with Meta-Learning,ICME 2023,2023,,,https://github.com/DefangChen/Knowledge-Distillation-Paper
689,Customizing Synthetic Data for Data-Free Student Learning,ICME 2023,2023,,,https://github.com/DefangChen/Knowledge-Distillation-Paper
690,Holistic Weighted Distillation for Semantic Segmentation,ICME 2023,2023,,,https://github.com/DefangChen/Knowledge-Distillation-Paper
691,Accelerating Diffusion Sampling with Classifier-based Feature Distillation,ICME 2023,2023,,,https://github.com/DefangChen/Knowledge-Distillation-Paper


In [410]:
pd.set_option('display.max_colwidth', None)
awesome_nlp_paper_df

,Title,Journal/Conference,Year,Author,ID/Link,Source
0,FitNets: Hints for Thin Deep Nets,ICLR 2015,2015,,,https://github.com/DefangChen/Knowledge-Distillation-Paper
1,Harnessing deep neural networks with logical rules,ACL 2016,2016,,,https://github.com/FLHonker/Awesome-Knowledge-Distillation
2,Unifying distillation and privileged information,ICLR 2016,2016,,,https://github.com/FLHonker/Awesome-Knowledge-Distillation
3,Face model compression by distilling knowledge from neurons,AAAI 2016,2016,,,https://github.com/FLHonker/Awesome-Knowledge-Distillation
4,Paying more attention to attention: Improving the performance of convolutional neural networks via attention transfer,ICLR 2017,2017,,,https://github.com/FLHonker/Awesome-Knowledge-Distillation
...,...,...,...,...,...,...
138,Cross-Task Knowledge Distillation in Multi-Task Recommendation,AAAI 2022,2022,,,https://github.com/FLHonker/Awesome-Knowledge-Distillation
139,Revisiting Label Smoothing and Knowledge Distillation Compatibility: What was Missing?,ICML 2022,2022,"Keshigeyan Chandrasegaran, Ngoc-Trung Tran, Yunqing Zhao, Ngai-Man Cheung",https://proceedings.mlr.press/v162/chandrasegaran22a/chandrasegaran22a.pdf,https://github.com/lhyfst/knowledge-distillation-papers
140,Consistency Models,ICML 2023,2023,,,https://github.com/DefangChen/Knowledge-Distillation-Paper
141,Curriculum Temperature for Knowledge Distillation,AAAI 2023,2023,"Zheng Li, Xiang Li, Lingfeng Yang, Borui Zhao, Renjie Song, Lei Luo, Jun Li, Jian Yang, AAAI 2023",https://arxiv.org/abs/2211.16231,https://github.com/lhyfst/knowledge-distillation-papers


In [411]:
awesome_paper_df.to_excel('awesome_paper_df.xlsx')
awesome_nlp_paper_df.to_excel('awesome_nlp_paper_df.xlsx')

### Reference

<b>Github</b>
<br>[DefangChen/Knowledge-Distillation-Paper](https://github.com/DefangChen/Knowledge-Distillation-Paper)
<br>[FLHonker/Awesome-Knowledge-Distillation](https://github.com/FLHonker/Awesome-Knowledge-Distillation)
<br>[lhyfst/knowledge-distillation-papers](https://github.com/lhyfst/knowledge-distillation-papers)